# Mini Project: Build a Machine Learning Model

## Predict Total Fare on the NYC Taxi Dataset

Welcome to the NYC Taxi Fare Prediction project! In this Colab, we will continue using the NYC Taxi Dataset to predict the fare amount for taxi rides using a subset of available features. We will go through three main stages: building a baseline model, creating a full model, and performing hyperparameter tuning to enhance our predictions.

Now that you've completed exploratory data analysis on this dataset you should have a good understanding of the feature space.

## Project Objectives

The primary objectives of this project are as follows:

Baseline Model: We will start by building a simple baseline model to establish a benchmark for our predictions. This model will serve as a starting point to compare the performance of our subsequent models.

Full Model: Next, we will develop a more comprehensive model that leverages machine learning techniques to improve prediction accuracy. We will use Scikit-Learn's model pipeline to build a framework that enables rapid experimentation.

Hyperparameter Tuning: Lastly, we will optimize our full model by fine-tuning its hyperparameters. By systematically adjusting the parameters that control model behavior, we aim to achieve the best possible performance for our prediction task.

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from datetime import datetime, timedelta

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# to install pyarrow on a modern python system running on Ubuntu/Debian, have to pass pip the --break-system-packages flag because there's no apt pkg for it (yet)
import pyarrow.parquet as pq

Load the NYC taxi dataset into a Pandas DataFrame and do a few basic checks to ensure the data is loaded properly. Note, there are several months of data that can be used. For simplicity, use the Yellow Taxi 2022-01 parquet file [here](https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet). Here are your tasks:

  1. Load the `yellow_tripdata_2022-01.parquet` file into Pandas.
  2. Print the first 5 rows of data.
  3. Drop any rows of data that contain NULL values.
  4. Create a new feature, 'trip_duration' that captures the duration of the trip in minutes.
  5. Create a varible named 'target_variable' to store the name of the thing we're trying to predict, 'total_amount'.
  6. Create a list called 'feature_cols' containing the feature names that we'll be using to predict our target variable. The list should contain 'VendorID', 'trip_distance', 'payment_type', 'PULocationID', 'DOLocationID', and 'trip_duration'.

In [53]:
# Load the dataset into a pandas DataFrame (from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
taxidata = pq.read_table('yellow_tripdata_2022-01.parquet').to_pandas()
taxidata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [54]:
# Display the first few rows of the dataset
taxidata.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [55]:
# Count the missing values
taxidata.isna().sum()

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          71503
trip_distance                0
RatecodeID               71503
store_and_fwd_flag       71503
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71503
airport_fee              71503
dtype: int64

In [56]:
# Drop rows with incorrect types or missing values.
taxidata.replace([np.inf, -np.inf], np.nan, inplace=True)
taxidata.dropna(inplace=True)


In [57]:
# Create new feature, 'trip_duration'.
#trip_duration = []
#trip_delta = timedelta()
#for i, row in taxidata.iterrows():
#    trip_delta = row['tpep_dropoff_datetime'] - row['tpep_pickup_datetime']
#    trip_duration.append(round(trip_delta.total_seconds()/60.0,2))
#taxidata['trip_duration'] = trip_duration

# doing it with list comprehension takes 3 seconds vs 2 minutes using a for loop above
a = taxidata['tpep_dropoff_datetime'] - taxidata['tpep_pickup_datetime']
taxidata['duration_min'] = [ round(x.total_seconds()/60.0, 2) for x in a ]

print(f"duration of first trip in data set: {taxidata['duration_min'][0]}")


duration of first trip in data set: 17.82


In [58]:
# Additional codeblock to add day of week and time of day, since we'll be getting rid of the actual timestamps for the models
# future self note : using list comprehension like this runs in about 4 seconds, versus a minute and half for iterrows()
taxidata['pickup_hour'] = [ x.hour for x in taxidata['tpep_pickup_datetime']]
taxidata['pickup_dayofweek'] = [x.weekday() for x in taxidata['tpep_pickup_datetime']]
print(taxidata['pickup_dayofweek'][0])

5


In [59]:
# Create a list called feature_col to store column names
feature_col = list(taxidata.columns)
feature_col

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee',
 'duration_min',
 'pickup_hour',
 'pickup_dayofweek']

Splitting a dataset into training and test sets is a crucial step in machine learning model development. It allows us to evaluate the performance and generalization ability of our models accurately. The training set is used to train the model, while the test set serves as an independent sample for evaluating its performance.

1. **Model Training**: The training set is used to fit the model, allowing it to learn the underlying patterns and relationships between the features and the target variable. By exposing the model to a diverse range of examples in the training set, it can capture the underlying structure of the data.

2. **Model Evaluation**: The test set, which is independent of the training set, is crucial for evaluating how well the trained model generalizes to unseen data. It provides an unbiased assessment of the model's performance on new instances. By measuring the model's accuracy, precision, recall, or other evaluation metrics on the test set, we can estimate how well the model will perform on unseen data.

3. **Preventing Overfitting**: Overfitting occurs when a model learns the training data's noise and idiosyncrasies instead of the underlying patterns. By evaluating the model on the test set, we can identify if the model is overfitting. If the model performs significantly worse on the test set compared to the training set, it indicates overfitting. In such cases, we might need to adjust the model, feature selection, or regularization techniques to improve generalization.

4. **Hyperparameter Tuning**: Splitting the dataset allows us to perform hyperparameter tuning on the model. Hyperparameters are configuration settings that control the learning process, such as learning rate, regularization strength, or the number of hidden layers in a neural network. By using a validation set (often created from a portion of the training set), we can iteratively adjust the hyperparameters and select the best combination that maximizes the model's performance on the validation set. The final evaluation on the test set provides an unbiased estimate of the model's performance.

By splitting the dataset into training and test sets, we can ensure that our models are both well-trained and accurately evaluated. This separation helps us understand how the model will perform on new, unseen data, which is critical for assessing its effectiveness and making informed decisions about its deployment.

Here is your task:

  1. Use Scikit-Learn's [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split the data into training and test sets. Don't forget to set the random state.

In [60]:
# Set up the target and training data set
target = taxidata['total_amount'].values
features_names = 'passenger_count trip_distance payment_type fare_amount airport_fee duration_min pickup_hour pickup_dayofweek'.split()
X = taxidata[features_names].copy()

print(target.shape)
print(X.shape) 
X.head()

(2392428,)
(2392428, 8)


,passenger_count,trip_distance,payment_type,fare_amount,airport_fee,duration_min,pickup_hour,pickup_dayofweek
0,2.0,3.80,1,14.5,0.0,17.82,0,5
1,1.0,2.10,1,8.0,0.0,8.40,0,5
2,1.0,0.97,1,7.5,0.0,8.97,0,5
3,1.0,1.09,2,8.0,0.0,10.03,0,5
4,1.0,4.30,1,23.5,0.0,37.53,0,5


In [61]:
# Split dataset into training and test sets
# stratify is giving me an error here, which I've never seen before
# subset of the data for modeling - need floats, so drop everything not a float & the target
X_train, X_test, y_train, y_test = train_test_split(X, target, train_size=0.8, random_state=42 )
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1913942, 8)
(1913942,)
(478486, 8)
(478486,)


The importance of a baseline model, even if it uses a simple strategy like always predicting the mean, cannot be understated. Here's why a baseline model is valuable:

1. **Performance Comparison**: A baseline model serves as a reference point for evaluating the performance of more sophisticated models. By establishing a simple yet reasonable baseline, we can determine whether our advanced models offer any significant improvement over this basic approach. It helps us set realistic expectations and gauge the effectiveness of our efforts.

2. **Model Complexity**: A baseline model provides insight into the complexity required to solve the prediction task. If a simple strategy like predicting the median performs reasonably well, it suggests that the problem might not necessitate complex modeling techniques. Conversely, if the baseline model performs poorly, it indicates the presence of more intricate patterns that need to be captured by more sophisticated models.

3. **Minimum Performance Requirement**: A baseline model can establish a minimum performance requirement for a predictive task. If we cannot outperform the baseline, it suggests that our models have failed to capture even the most fundamental relationships within the data. In such cases, we may need to revisit our data preprocessing steps, feature engineering techniques, or consider other external factors affecting the task.

4. **Identifying Data Issues**: A baseline model can help identify potential issues within the dataset. If the baseline model performs poorly, it may indicate problems like missing values, outliers, or data inconsistencies. These issues can be further investigated and resolved to improve the overall model performance.

While a baseline model like always predicting the median may not offer the highest prediction accuracy, its importance lies in its role as a starting point for model development and evaluation. It provides a solid foundation for comparing and assessing the performance of more complex models, ensuring that any improvements made are meaningful and significant.

Here is your task:

  1. Create a model that always predicts the mean total fare of the training dataset. Use Scikit-Learn's [mean_absolute_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) to evaluate this model. Is it any good?

In [62]:
# Create a baseline for mean absolute error of total amount
lr = LinearRegression(n_jobs=-1, copy_X=True)
#lr_param_grid = {'fit_intercept': (True, False), 'positive': (True, False)}
#baseline_gs = GridSearchCV(estimator=lr, param_grid=lr_param_grid, n_jobs=-1)
lr.fit(X_train, y_train)


LinearRegression(n_jobs=-1)

In [63]:
#y_pred = baseline_gs.pred(X_test)
# baseline_me = baseline_gs.score(y_test, y_pred)
lr_y_pred = lr.predict(X_test)
lr_mae_score = mean_absolute_error(y_test, lr_y_pred)

print(f"baseline linear model score = {round(lr_mae_score,4)}")

baseline linear model score = 1.402


With a baseline metric in place, we can try to build a machine learning model. Obviously, if the model can't beat the baseline then there are some major issues to be resolved.

It's always a good idea to start with a simple machine learning model, like linear regression, and build upon it if necessary.

Here are your tasks:

  1. Use Scikit-Learn's [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) to preprocess the categorical and continuous features independently. Apply the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to the continuous columns and [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to the categorical columns.

  One-hot encoding is a popular technique used to represent categorical variables numerically in machine learning models. It transforms categorical features into a binary vector representation, where each category is represented by a binary column. Here's an explanation of one-hot encoding:

  When working with categorical variables, such as colors (e.g., red, blue, green) or vehicle types (e.g., car, truck, motorcycle), machine learning algorithms often require numerical inputs. However, directly assigning numerical values to categories can introduce unintended relationships or orderings between them. For example, assigning the values 0, 1, and 2 to the categories red, blue, and green may imply a sequential relationship, which is not desired.

  One-hot encoding solves this problem by creating new binary columns, equal to the number of unique categories in the original feature. Each binary column represents a specific category and takes a value of 1 if the data point belongs to that category, and 0 otherwise. This encoding ensures that no implicit ordering or relationship exists between the categories.

  2. Integrate the preprocessor in the previous step with Scikit-Learn's [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) model using a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

  3. Train the pipeline on the training data.

  4. Evaluate the model using mean absolute error as a metric on the test data. Does the model beat the baseline?


In [64]:
# Use Scikit-Learn's ColumnTransformer to preprocess the categorical and
# continuous features independently.
# First the continuous; Note: default fit_transform returns a numpy array, so this keeps it in a pandas DataFrame

numeric_features = 'passenger_count trip_distance fare_amount airport_fee duration_min'.split()
scaler = StandardScaler() # if doing it manually (not in a pipeline), it's convenient to use .set_output(transform="pandas")

# Second the categorical
# Note: The only original categorical variable I can see is "store_and_fwd_flag' which is Y/N column.
# It was dropped in the previous Linear model baseline because it's an object(string) and needs to be encoded as a float.
# alternatively, could use LabelEncoder for a binary column like this
# I also added 2 additional categoricals from feature analysis / EDA for day of week and hour of day
categorical_features = ['payment_type', 'pickup_hour','pickup_dayofweek']
ohe = OneHotEncoder(drop='first')

# Python datetime uses Monday = 1 for day of week, so OneHotEncoder will drop the last column, which is Monday
#weekday_labels = 'Tue Wed Thu Fri Sat Sun'.split()
#dow_headers = ['dow_' + i for i in weekday_labels]

preproc = ColumnTransformer( transformers = [
    ('num', scaler, numeric_features),
    ('cat', ohe, categorical_features)],
    verbose = True,
    n_jobs = -1
)

print(preproc)

ColumnTransformer(n_jobs=-1,
                  transformers=[('num', StandardScaler(),
                                 ['passenger_count', 'trip_distance',
                                  'fare_amount', 'airport_fee',
                                  'duration_min']),
                                ('cat', OneHotEncoder(drop='first'),
                                 ['payment_type', 'pickup_hour',
                                  'pickup_dayofweek'])],
                  verbose=True)


In [65]:
# Create a pipeline object containing the column transformations and regression
pipe = Pipeline( steps=[
    ('preprocessor',preproc),
    ('classifier', LinearRegression(copy_X = True) )],
    verbose = True
)

print(pipe)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num', StandardScaler(),
                                                  ['passenger_count',
                                                   'trip_distance',
                                                   'fare_amount', 'airport_fee',
                                                   'duration_min']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['payment_type',
                                                   'pickup_hour',
                                                   'pickup_dayofweek'])],
                                   verbose=True)),
                ('classifier', LinearRegression())],
         verbose=True)


In [66]:
# Fit the pipeline on the training data.
X_train, X_test, y_train, y_test = train_test_split(X, target, train_size=0.8, random_state=42 )

pipe.fit(X_train, y_train)
X_train.head()

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   3.5s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   2.8s


,passenger_count,trip_distance,payment_type,fare_amount,airport_fee,duration_min,pickup_hour,pickup_dayofweek
2156043,1.0,2.57,2,14.5,0.0,21.40,13,4
1279349,1.0,2.83,1,14.5,0.0,19.67,13,1
2008041,1.0,3.26,2,13.5,0.0,15.42,20,2
880165,2.0,0.37,2,6.5,0.0,8.47,12,3
1519283,1.0,1.51,1,7.5,0.0,8.28,9,4


In [67]:
# Make predictions on the test data.
y_pred_pipe = pipe.predict(X_test)
score_pipe_lr = mean_absolute_error(y_test, y_pred_pipe)
print(f'Pipelined Linear Regression MSE: {score_pipe_lr:0.3f}')

Pipelined Linear Regression MSE: 1.371


Random Forest Regression and Linear Regression are two commonly used regression algorithms, each with its own advantages and suitability for different scenarios. Random Forest Regression offers several advantages over Linear Regression, including:

1. **Non-linearity**: Random Forest Regressor is capable of capturing non-linear relationships between features and the target variable. In contrast, Linear Regression assumes a linear relationship between the features and the target. When faced with non-linear relationships or complex feature interactions, Random Forest Regressor can provide more accurate predictions.

2. **Robustness to Outliers**: Random Forest Regressor is generally more robust to outliers compared to Linear Regression. Outliers can disproportionately impact the coefficients and predictions of Linear Regression models. However, as an ensemble of decision trees, Random Forest Regressor can mitigate the effect of outliers by averaging predictions from multiple trees.

3. **Feature Importance**: Random Forest Regressor provides a measure of feature importance, which helps identify the most influential features for making predictions. This information is useful for feature selection, understanding the underlying relationships in the data, and gaining insights into the problem domain. Unlike Linear Regression, which provides coefficient values indicating the direction and magnitude of relationships, Random Forest Regressor explicitly highlights feature importance.

4. **Handling of Categorical Variables**: Random Forest Regressor can effectively handle categorical variables without requiring pre-processing steps like one-hot encoding. It can directly incorporate categorical variables into the model, making it more convenient when working with mixed data types. In contrast, Linear Regression often requires categorical variables to be encoded or transformed before use.

5. **Handling of High-Dimensional Data**: Random Forest Regressor can handle datasets with a large number of features (high dimensionality) by automatically selecting subsets of features during the construction of individual decision trees. This reduces the risk of overfitting, which is a concern with Linear Regression when dealing with high-dimensional data.

6. **Resistance to Multicollinearity**: Random Forest Regressor is less affected by multicollinearity, which occurs when predictor variables are highly correlated. In Linear Regression, highly correlated features can lead to unstable coefficient estimates, making it challenging to interpret the individual effects of each feature. Random Forest Regressor, as an ensemble approach, is less impacted by multicollinearity because each tree is built independently.

Here are your tasks:

  1. Build a Random Forest Regressor model using Scikit-Learn's [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) and train it on the train data.

  2. Evaluate the performance of the model on the test data using mean absolute error as a metric. Mess around with various input parameter configurations to see how they affect the model. Can you beat the performance of the linear regression model?

In [68]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_train.columns)
print(X_test.columns)


(1913942, 8)
(478486, 8)
(1913942,)
(478486,)
Index(['passenger_count', 'trip_distance', 'payment_type', 'fare_amount',
       'airport_fee', 'duration_min', 'pickup_hour', 'pickup_dayofweek'],
      dtype='object')
Index(['passenger_count', 'trip_distance', 'payment_type', 'fare_amount',
       'airport_fee', 'duration_min', 'pickup_hour', 'pickup_dayofweek'],
      dtype='object')


In [69]:
# Build random forest regressor model with some chosen hyperparameters
rf_test = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=20, min_samples_leaf=15, min_weight_fraction_leaf=0.15, min_samples_split=16)
rf_test.fit(X_train, y_train)
rf_test

RandomForestRegressor(max_depth=20, min_samples_leaf=15, min_samples_split=16,
                      min_weight_fraction_leaf=0.15, random_state=42)

In [70]:
# Make predictions on the test data
y_pred_rf_test = rf_test.predict(X_test)
rf_test_mse = mean_absolute_error(y_test, y_pred_rf_test)
print(f"First: Random Forest MSE: {rf_test_mse:0.3f}")

First: Random Forest MSE: 3.920


Hyperparameter tuning plays a critical role in machine learning model development. It involves selecting the optimal values for the hyperparameters, which are configuration settings that control the behavior of the learning algorithm. Here's why hyperparameter tuning is so important in ML:

1. **Optimizing Model Performance**: The choice of hyperparameters can significantly impact the model's performance. By fine-tuning the hyperparameters, we can improve the model's accuracy, precision, recall, or other performance metrics. It helps to extract the maximum predictive power from the chosen algorithm and ensures that the model is well-suited to the specific problem at hand.

2. **Avoiding Overfitting and Underfitting**: Hyperparameter tuning helps strike a balance between overfitting and underfitting.

3. **Exploring Model Complexity**: Hyperparameter tuning enables us to explore the complexity of the model. For instance, in algorithms like decision trees or neural networks, we can adjust the number of layers, the number of neurons, or the maximum depth of the tree. By systematically modifying these hyperparameters, we can understand how different levels of complexity impact the model's performance and find the right balance between simplicity and complexity.

Note, there are multiple approaches to hyperparemeter tuning.  

While grid search is the easiest to understand and implement there are many advantages of Bayesian search over grid search for hyperparameter tuning:

1. **Efficiency**: Bayesian search is generally more efficient than grid search. Grid search explores all possible combinations of hyperparameter values, which can be computationally expensive and time-consuming, especially when dealing with a large number of hyperparameters or a wide range of values. Bayesian search, on the other hand, intelligently selects the next hyperparameter configuration to evaluate based on the results of previous evaluations. It focuses on areas of the hyperparameter space that are more likely to yield better performance, reducing the number of evaluations needed.

2. **Flexibility**: Bayesian search is flexible in handling continuous and discrete hyperparameters. It can handle both types of hyperparameters naturally and effectively. In contrast, grid search is more suitable for discrete hyperparameters but may struggle with continuous ones, as it requires discretization or defining a finite set of values to search over.

3. **Adaptive Search**: Bayesian search adapts its search strategy based on the results of previous evaluations. It maintains a probability distribution over the hyperparameter space, updating it with each evaluation. This allows it to dynamically allocate more evaluations to promising regions and explore unexplored areas. In contrast, grid search follows a fixed and predefined search grid, regardless of the results of previous evaluations.

4. **Better Convergence**: Bayesian search has the potential to converge to the optimal hyperparameter configuration more quickly.

Here are your tasks:

  1. Perform a grid-search on a Random Forest Regressor model. Only search the space for the parameters 'n_estimators', 'max_depth', and 'min_samples_split'. Note, this can take some time to run. Make sure you set reasonable boundaries for the search space. Use Scikit-Learn's [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) method.

  2. After you've identified the best parameters, train a random forest regression model using these parameters on the full training data.

  3. Evaluate the model from the previous step using the test data. How does your model perform?

In [71]:
# Define the hyperparameters to tune.

rf_params = {
    'n_estimators':np.arange(start=50,stop=300, step=50), 
    'max_depth':np.arange(start=2,stop=10, step=2),
    'min_samples_split':np.arange(start=2,stop=10, step=2)
}


In [72]:
# Perform grid search to find the best hyperparameters. This could take a while.
# this took 8hr30min
rf = RandomForestRegressor(random_state=42)
grid_rf = GridSearchCV(estimator=rf, param_grid=rf_params, verbose=True, n_jobs=-1)
grid_rf.fit(X_train, y_train)


Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': array([2, 4, 6, 8]),
                         'min_samples_split': array([2, 4, 6, 8]),
                         'n_estimators': array([ 50, 100, 150, 200, 250])},
             verbose=True)

In [75]:
# Get the best model and its parameters.
# best params indicates:
# grid_rf.best_params_
# {'max_depth': np.int64(8),
# 'min_samples_split': np.int64(4),
# 'n_estimators': np.int64(100)}

best_rf = grid_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)
best_rf_score = mean_absolute_error(y_test, y_pred_rf)
print(f"Grid Search: Best Random Forest Accuracy MSE: {best_rf_score:0.3f}")


Grid Search: Best Random Forest Accuracy MSE: 1.176


{'max_depth': np.int64(8),
 'min_samples_split': np.int64(4),
 'n_estimators': np.int64(100)}

In [77]:
# Fit the best classifier on the training data.
best_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=np.int64(8), min_samples_split=np.int64(4),
                      n_estimators=np.int64(100), random_state=42)

In [78]:
# Make predictions on the test data
y_pred_final = best_rf.predict(X_test)
final_rf_score = mean_absolute_error(y_test, y_pred_final)
print(f"Grid Search: Final Random Forest Accuracy MSE: {final_rf_score:0.3f}")


Grid Search: Final Random Forest Accuracy MSE: 1.176
